In [6]:
import quantumpropagator as qp
import matplotlib.pyplot as plt
import matplotlib
from ipywidgets import interact,fixed #, interactive, fixed, interact_manual
import ipywidgets as widgets

import os
import sys
import glob
import pandas as pd
import numpy as np

%matplotlib ipympl

plt.rcParams.update({'font.size': 10})

In [7]:
fol = '/home/alessio/w-August-Run/'
outputFolder = '/home/alessio/w-August-Run/OUTPUTS'
subfolders2 = sorted([dir for dir in os.listdir(fol) if os.path.isdir(os.path.join(fol,dir))])
subfolders =  [ dir for dir in subfolders2 if dir not in ['HTML','csv','OUTPUTS'] ]
print(''.join(['{} -> {}\n'.format(a,b) for a,b in enumerate(subfolders)]))

0 -> b-UV-0.22_0000
1 -> m-IR-Polarized-INV_0000
2 -> m-IR-Polarized-LON_0000
3 -> m-IR-Polarized-NOR_0000
4 -> position-from1_0000
5 -> position-from2_0000
6 -> position-from3_0000
7 -> position-from4_0000
8 -> position-from5_0000
9 -> position-from6_0000
10 -> t-FC_excited_phi_0000
11 -> t-IR-to-excite-phi_0000
12 -> t-UV-0.22_excited_phi_0001
13 -> x-SubMask_from_IR_0000
14 -> x-SubMask_from_IR_INV_0000
15 -> z-from1_0000



In [8]:

def process_folder(this_folder,fol,outputFolder):
    '''
    this function will create a correct set of files to give to Francoise
    '''
    project_folder = os.path.join(fol,this_folder)
    project_output = os.path.join(outputFolder,this_folder)
    output_norm = os.path.join(project_folder, 'output')
    output_popu = os.path.join(project_folder, 'outputPopul')
    output_abso = os.path.join(project_folder, 'Output_Abs')
    output_regi = os.path.join(project_folder, 'Output_Regions')
    output_dipo = os.path.join(project_folder, 'Output_Dipole')
    
    df_norm2 = pd.read_csv(output_norm, delim_whitespace=True, index_col=0, names=['counter', 'steps', 'fs','Norm deviation','Kinetic','Potential','Total','Total Deviation','Xpulse','Ypulse','Zpulse','AbZino'])
    df_popu2 = pd.read_csv(output_popu, delim_whitespace=True, names=['fs_1', 'S0', 'S1','S2','S3','S4','S5','S6','S7'])
    df_abso2 = pd.read_csv(output_abso, delim_whitespace=True, names=['Time AU', 'Abs Tot', 'Abs S0', 'Abs S1','Abs S2','Abs S3','Abs S4','Abs S5','Abs S6','Abs S7'])
    df_regi2 = pd.read_csv(output_regi, delim_whitespace=True, names=['FC','Reactants','Products'])
    df_dipo2 = pd.read_csv(output_dipo, delim_whitespace=True, names=['fs_2','dipX','dipY','dipZ'])
    
    # I need to cut down to different file size <- this will become obsolete
    dfs = [df_norm2,df_popu2,df_abso2,df_regi2,df_dipo2]
    lengths = min([x.shape[0] for x in dfs])
    
    df_norm, df_popu, df_abso, df_regi, df_dipo = [ x.drop(x.index[lengths:]) for x in dfs ]
    
    df_all = pd.concat([df_norm, df_popu, df_abso, df_regi, df_dipo],axis=1)
    
    # time step in AU
    AU_dt = df_abso['Time AU'].iloc[1]
    
    # create the absorbed columns
    for statZ in ['Tot','S0','S1','S2','S3','S4','S5','S6','S7']:
        df_all['P(t) {}'.format(statZ)] = -(np.cumsum(df_all['Abs {}'.format(statZ)])*AU_dt)
    
    # change sign of norm
    df_all['Norm deviation'] = -df_all['Norm deviation']
    
    # clean the paper
    final_df = df_all.drop(['Abs Tot', 'Abs S0', 'Abs S1','Abs S2','Abs S3','Abs S4','Abs S5','Abs S6','Abs S7','fs_1','fs_2','AbZino'],axis=1)
    
    csv_name = '{}.csv'.format(project_output)
    final_df.to_csv(csv_name)
    
process_folder(subfolders[0],fol,outputFolder)

/home/alessio/config/miniconda/envs/quantumpropagator/lib/python3.5/site-packages/pandas/core/indexes/api.py:77: RuntimeWarning: unorderable types: tuple() < int(), sort order is undefined for incomparable objects
  result = result.union(other)


TypeError: object of type 'int' has no len()

In [5]:
betweenbondsfolder = '/home/alessio/densities/From_pinot/calculationZ'